In [ ]:
from langchain.schema import Document
from vertexai.language_models import TextEmbeddingModel
from opensearchpy import OpenSearch
from google.cloud import aiplatform

def buscar_documentos_en_opensearch(pregunta: str, index_name="topicosindex", top_k=3) -> list[Document]:

    # Inicialización (puedes ponerla en tu setup global)
    aiplatform.init(project="mestria-puj-s2", location="us-central1")
    embedding_model = TextEmbeddingModel.from_pretrained("text-multilingual-embedding-002")

    client = OpenSearch(
        hosts=[{"host": "localhost", "port": 9200}],
        http_auth=("admin", "Nagato123!"),
        use_ssl=True,
        verify_certs=False
    )

    # Embedding de la pregunta
    embedding = embedding_model.get_embeddings([pregunta])[0].values

    # Consulta KNN a OpenSearch
    query = {
        "size": top_k,
        "query": {
            "knn": {
                "embedding": {
                    "vector": embedding,
                    "k": top_k
                }
            }
        }
    }

    resultado = client.search(index=index_name, body=query)

    # Convertir a lista de Document
    documentos = []
    for hit in resultado["hits"]["hits"]:
        doc = Document(
            page_content=hit["_source"]["text"],
            metadata={
                "titulo": hit["_source"]["metadata"]["title"],
                "departamento": hit["_source"]["metadata"]["keyword"],
                "id_documento": hit["_source"]["metadata"]["id_documento"],
                "score": hit["_score"]
            }
        )
        documentos.append(doc)

    return documentos


/opt/conda/envs/topicos_env/lib/python3.10/site-packages/opensearchpy/connection/http_urllib3.py:214: UserWarning: Connecting to https://localhost:9200 using SSL with verify_certs=False is insecure.
  warnings.warn(


In [7]:
buscar_documentos_en_opensearch("¿Dónde se celebra un carnaval famoso en Colombia?")



/opt/conda/envs/topicos_env/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


[Document(metadata={'titulo': 'Dato curioso sobre Atlántico', 'departamento': 'Atlántico', 'id_documento': 4, 'score': 0.78385055}, page_content='En Barranquilla se celebra uno de los carnavales más importantes del mundo.'),
 Document(metadata={'titulo': 'Dato curioso sobre Bolívar', 'departamento': 'Bolívar', 'id_documento': 5, 'score': 0.59251076}, page_content='Cartagena, en Bolívar, fue uno de los puertos más importantes del imperio español en América.'),
 Document(metadata={'titulo': 'Dato curioso sobre Amazonas', 'departamento': 'Amazonas', 'id_documento': 1, 'score': 0.5869829}, page_content='El Amazonas colombiano alberga parte del mayor bosque tropical continuo del mundo.')]

In [4]:
from agent.invoker import IAInvoker
from langchain.schema import HumanMessage, AIMessage

# Historial acumulado por el frontend o por tu lógica de sesión
chat_history = [
    HumanMessage(content="¿Qué es el machine learning?"),
    AIMessage(content="Es una subárea de la inteligencia artificial que..."),
]

invoker = IAInvoker(project_id="mestria-puj-s2")

# Paso siguiente de la conversación
respuesta = invoker.run("¿Y qué relación tiene con los datos?", chat_history)
print(respuesta)

/opt/conda/envs/topicos_env/lib/python3.10/site-packages/opensearchpy/connection/http_urllib3.py:214: UserWarning: Connecting to https://localhost:9200 using SSL with verify_certs=False is insecure.
  warnings.warn(
/opt/conda/envs/topicos_env/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Según la información proporcionada, los documentos hablan sobre datos curiosos de departamentos de Colombia como Bolívar, Amazonas y Arauca.

No se menciona la relación entre machine learning y los datos en este contexto.
